In [ ]:
# Softmax classifier for guessing minesweeper board position and whether it has a mine or not

In [1]:
# Import libraries for simulation
import tensorflow as tf
import numpy as np
import random as r
import datetime as dt

/home/ruben/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
dimensions = (12,12)
mineProbability = 0.16      # Probability that a square contain a mine

In [3]:
# Clears a square on the minesweeper board.
# If it had a mine, return true
# Otherwise if it has no adjacent mines, recursively run on adjacent squares
# Return false
def clearSquare(board,adjacency,row,col):
    if board[row,col] == 1:
        return True
    if adjacency[row,col] >= 0:
        return False
    n = 0
    for r in range(row-1,row+2):
        for c in range(col-1,col+2):
            if 0 <= r and r < rows and 0 <= c and c < cols:
                n += board[r,c]
    adjacency[row,col] = n
    if n == 0:
        for r in range(row-1,row+2):
            for c in range(col-1,col+2):
                if 0 <= r and r < rows and 0 <= c and c < cols:
                    clearSquare(board,adjacency,r,c)
    return False

In [4]:
# This takes a mine board and gives a mine count with mines removed, and other random squares removed
def boardPartialMineCounts(board):
    clearProbability = r.uniform(0.05,0.5)
    result = np.full(dimensions,-1)
    for index, x in np.ndenumerate(board):
        row,col = index
        if not(x) and result[row,col] == -1 and r.uniform(0,1) < clearProbability:
            clearSquare(board,result,row,col)
    return result

In [5]:
# Generates a random training batch of size at most n
def next_training_batch(n):
    batch_xs = []
    batch_ys = []
    boards = []
    for _ in range(n):
        board = np.random.random(dimensions) < mineProbability
        counts = boardPartialMineCounts(board)
        validGuesses = np.append(((counts == -1).astype(int) - board).flatten().astype(float),
                                 board.flatten().astype(float))
        validGuessesSum = sum(validGuesses)
        if validGuessesSum > 0:
            # encode counts as one hot
            countsOneHot = np.zeros((counts.size,10))
            countsOneHot[np.arange(counts.size), counts.flatten() + 1] = 1
            batch_xs.append(countsOneHot.flatten())
            batch_ys.append(validGuesses / validGuessesSum)
            boards.append(board)
    return (np.asarray(batch_xs), np.asarray(batch_ys), boards)

In [6]:
# Create the model
rows, cols = dimensions
size = rows*cols
mineCountsOneHot = tf.placeholder(tf.float32, [None, size*10], name="mineCountsOneHot")
#mineCountsOneHot = tf.reshape(tf.one_hot(mineCounts+1,10), [-1, size*10])
W = tf.Variable(tf.random_normal([size*10, size*2], stddev=0.01), name="W")
b = tf.Variable(tf.random_normal([size*2], stddev=0.01), name="b")
y = tf.matmul(mineCountsOneHot, W) + b

In [7]:
validGuessAverages = tf.placeholder(tf.float32, [None, size*2], name="validGuessAverages")

In [8]:
# Loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=validGuessAverages, logits=y))

In [9]:
# Summaries for tensorboard
with tf.name_scope('W_reshape'):
    image_shaped_W = tf.reshape(W, [-1, size*10, size*2, 1])
    tf.summary.image('W', image_shaped_W, 1000)

with tf.name_scope('b_reshape'):
    image_shaped_b = tf.reshape(b, [-1, rows*2, cols, 1])
    tf.summary.image('b', image_shaped_b, 1000)

_ = tf.summary.scalar('loss', cross_entropy)

In [10]:
# Optimiser
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

In [11]:
# Create session and initialise or restore stuff
savePath = './saves.tf.Mines6/' + str(dimensions) + '/'
saver = tf.train.Saver()

sess = tf.InteractiveSession()

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('.', sess.graph)

In [12]:
tf.global_variables_initializer().run()

In [13]:
# Restore model?
#saver.restore(sess, savePath + "model-8000")

In [ ]:
# Train
for iteration in range(10001):
    batch_xs, batch_ys, _ = next_training_batch(1000)
    summary, loss, _ = sess.run([merged, cross_entropy, train_step],
                                  feed_dict={mineCountsOneHot: batch_xs, validGuessAverages: batch_ys})
    writer.add_summary(summary, iteration)
    print('%s: Loss at step %s: %s' % (dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), iteration, loss))
    if iteration % 100 == 0:
        save_path = saver.save(sess, savePath + 'model', global_step=iteration)
        print("Model saved in file: %s" % save_path)

2017-11-07 14:18:04: Loss at step 0: 5.672
Model saved in file: ./saves.tf.Mines6/(12, 12)/model-0
2017-11-07 14:18:06: Loss at step 1: 5.6473
2017-11-07 14:18:08: Loss at step 2: 5.63212
2017-11-07 14:18:10: Loss at step 3: 5.61701
2017-11-07 14:18:12: Loss at step 4: 5.61179
2017-11-07 14:18:14: Loss at step 5: 5.6016
2017-11-07 14:18:15: Loss at step 6: 5.59099
2017-11-07 14:18:17: Loss at step 7: 5.57955
2017-11-07 14:18:19: Loss at step 8: 5.57101
2017-11-07 14:18:21: Loss at step 9: 5.56409
2017-11-07 14:18:23: Loss at step 10: 5.55115
2017-11-07 14:18:25: Loss at step 11: 5.53616
2017-11-07 14:18:26: Loss at step 12: 5.53234
2017-11-07 14:18:28: Loss at step 13: 5.52216
2017-11-07 14:18:30: Loss at step 14: 5.51348
2017-11-07 14:18:32: Loss at step 15: 5.50518
2017-11-07 14:18:34: Loss at step 16: 5.49984
2017-11-07 14:18:36: Loss at step 17: 5.49259
2017-11-07 14:18:38: Loss at step 18: 5.48465
2017-11-07 14:18:39: Loss at step 19: 5.4753
2017-11-07 14:18:41: Loss at step 20: 5

In [ ]:
# Test trained model on larger batch size
batch_xs, batch_ys, _ = next_training_batch(1000)
print(sess.run(cross_entropy, feed_dict={mineCountsOneHot: batch_xs, validGuessAverages: batch_ys}))

In [ ]:
# Run a test
batchSize = 10000
batch_xs, batch_ys, _ = next_training_batch(batchSize)

predictions = sess.run(tf.nn.softmax(y), feed_dict={mineCountsOneHot: batch_xs, validGuessAverages: batch_ys})
bestSquares = [pred.argmax() for pred in predictions]
unfrees = (batch_ys == 0).astype(int)
frees = [unfrees[i][bestSquares[i]] for i in range(batchSize)]
print("Number of errors for batch size of ", batchSize)
print(sum(frees))

In [ ]:
# Find boards that we failed on
batchSize = 1000
batch_xs, batch_ys, _ = next_training_batch(batchSize)

predictions = sess.run(tf.nn.softmax(y), feed_dict={mineCountsOneHot: batch_xs, validGuessAverages: batch_ys})
bestSquares = [pred.argmax() for pred in predictions]
unfrees = (batch_ys == 0).astype(int)
guesses = [unfrees[i][bestSquares[i]] for i in range(batchSize)]
for i in range(batchSize):
    if guesses[i] == 1:
        print(batch_xs[i].reshape(dimensions))
        summary = sess.run(tf.summary.image('mine_miss', tf.reshape((batch_xs[i]+1).astype(float),[-1,rows,cols,1]), 100))
        writer.add_summary(summary)

In [ ]:
#batch_xs = [[-1,1,-1,0,0,0,-1,-1,-1,1,1,1,-1,-1,1,2,2,1,1,-1,2,1,1,-1,-1,2,2,-1,-1,2,-1,1,1,0,-1,-1,2,-1,-1,4,-1,2,-1,1,2,-1,1,0,1,2,-1,3,2,2,1,-1,2,-1,1,0,0,1,1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,0,0,3,-1,4,1,2,-1,1,-1,-1,0,0,0,2,-1,-1,-1,2,-1,1,0,0,0,-1,1,2,-1,2,1,2,2,3,3,2,-1,-1,1,-1,1,-1,0,1,2,-1,-1,-1,1,1,1,-1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,1,-1,-1,-1]]
batch_xs0 = [-1] * (size)
batch_xs0[0] = 1
batch_xs0[1] = 1
batch_xs0[cols] = 1

predictions = sess.run(tf.nn.softmax(y), feed_dict={mineCountsOneHot: [batch_xs0]})
bestSquares = [pred.argmax() for pred in predictions]

print(bestSquares[0] // cols, bestSquares[0] % cols)

In [ ]:
np.save("./W", sess.run(W))

In [ ]:
np.save("./b", sess.run(b))

In [ ]:
np.savez("./model", sess.run([W,b]))